# Orekit Propagation with MSIS Using Predicted Space Weather Indices

This notebook uses forecasted space weather indices (ap_index_nT_pred and f10.7_index_pred) stored in `/data/sat_density_omni_forcasted` together with initial state data (from `initial_state_folder`) for a given file_id. Using these inputs, we propagate the orbit with Orekit and compute the atmospheric density using MSIS. The MSIS model is fed the predicted values instead of the original OMNI measurements.

The outputs (orbital elements and MSIS density) are then saved and/or plotted.


In [ ]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import orekit
from orekit.pyhelpers import setup_orekit_curdir
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants
from org.orekit.models.earth.atmosphere import PythonAtmosphere
import math

# Initialize Orekit and the JVM
vm = orekit.initVM()
setup_orekit_curdir()

# (Optionally set matplotlib parameters)
plt.rcParams.update({'font.size': 12})

# ------------------------------------------------------------------------------
# A custom MSIS model that uses the predicted values.
# ------------------------------------------------------------------------------
class PredictedMSIS(PythonAtmosphere):
    def __init__(self):
        super().__init__()
    
    def run_with_predicted(self, dt, lon, lat, alt, f107_pred, ap_pred):
        """
        Run MSIS using the provided predicted F10.7 and Ap values.
        
        Args:
            dt (datetime): Simulation datetime.
            lon (float): Longitude (degrees)
            lat (float): Latitude (degrees)
            alt (float): Altitude (km)
            f107_pred (float): Predicted F10.7 solar radio flux.
            ap_pred (float): Predicted Ap index.
        
        Returns:
            float: Density value from MSIS.
        """
        # For simplicity, here we pass the predicted values as scalars.
        # You might want to adjust how the "aps" array is built.
        result = msis.run(
            dates=[dt],
            lons=[lon],
            lats=[lat],
            alts=[alt],
            f107s=[f107_pred],
            aps=[[ap_pred]*7]  # replicate the Ap value (for 7 required inputs)
        )
        return result[0, 0]

# Import the pyMSIS msis function (make sure pymsis is installed)
from pymsis import msis


## 1. Load Forecasted Sat Density Data and Initial State

This cell loads the forecasted sat_density_omni_forcasted file for a given file_id and the corresponding initial state file from the initial_state_folder.


In [ ]:
# Define file_id (adjust as needed)
file_id = 1

# Load the forecasted sat density file (which has the predicted columns)
sat_forecasted_path = Path("./data/sat_density_omni_forcasted") / f"sat_density_{file_id:05d}.csv"
predicted_sat_df = pd.read_csv(sat_forecasted_path, parse_dates=["Timestamp"])
predicted_sat_df.sort_values("Timestamp", inplace=True)
print("Forecasted sat density file loaded. Sample:")
print(predicted_sat_df.head())

# Load the corresponding initial state file (assuming one file per file_id in initial_state_folder)
initial_state_path = Path("./data/initial_state") / f"initial_state_{file_id:05d}.csv"
initial_state_df = pd.read_csv(initial_state_path, parse_dates=["Timestamp"])
initial_state_df.sort_values("Timestamp", inplace=True)
print("Initial state file loaded. Sample:")
print(initial_state_df.head())


## 2. Setup Propagation and MSIS Using Predicted Space Weather Values

In this cell, we prepare the propagation using Orekit. We use the initial state from the corresponding file, and then at each desired timestamp (from the forecasted sat density file) we will use our custom MSIS model that is fed the predicted values.


In [ ]:
# For demonstration, we take the first row from the initial state file as our simulation initial state.
# You might need to adapt this based on your actual file structure.
initial_state = initial_state_df.iloc[0]

# Define a simulation point: for example, propagate to each timestamp in the forecast file.
# For simplicity, we will assume the propagation uses a fixed position (or use initial state position).
# In practice, you might want to compute a new orbit using Orekit's propagator.

# Here we assume a fixed location (latitude, longitude, altitude) for MSIS density computation.
# For example, let's assume a location at (lat=45 deg, lon=0 deg, alt=400 km).
lat = 45.0   # degrees
lon = 0.0    # degrees
alt = 400.0  # km

# For each forecast timestamp, we will:
# 1) Use the corresponding predicted ap_index_nT_pred and f10.7_index_pred.
# 2) Propagate (or simply compute MSIS density) at that timestamp.
# We'll store the density values in a new column.

from datetime import datetime

# Create an instance of our custom MSIS model (here we assume no extra data is needed)
msis_model = PredictedMSIS()

densities = []
for idx, row in predicted_sat_df.iterrows():
    # Get the forecast timestamp
    t = row["Timestamp"]
    # Convert t to a Python datetime object (if needed)
    dt = pd.to_datetime(t)
    
    # Get predicted values for this timestamp
    ap_pred = row["ap_index_nT_pred"]
    f107_pred = row["f10.7_index_pred"]
    
    # Run MSIS with predicted values.
    # Here we use our custom method; in practice, you might need to adjust parameters.
    density = msis_model.run_with_predicted(dt, lon, lat, alt, f107_pred, ap_pred)
    densities.append(density)

# Append densities to the predicted_sat_df.
predicted_sat_df["MSIS_density"] = densities

print("Propagation complete. Sample of results:")
print(predicted_sat_df.head())


## 3. Plot Propagation Results

This cell plots the computed MSIS density over the forecast timestamps.


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(12, 6))
plt.plot(predicted_sat_df["Timestamp"], predicted_sat_df["MSIS_density"], marker='o', linestyle='-')
plt.xlabel("Timestamp")
plt.ylabel("MSIS Density")
plt.title("MSIS Density Computed with Predicted ap_index_nT and f10.7_index")
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M:%S'))
plt.tight_layout()
plt.show()
